In [36]:
import nltk
import os
#chat = open("../_chat.txt", encoding="utf-8")
with open("../_chat.txt",  encoding="utf-8") as f:
    chat = [line.strip('\n') for line in f]

In [105]:
#print(chat[:500])
import re
joinedChat=' '.join(chat)
print(joinedChat[:500])
pattern=re.compile(r"\[[^\]]*\] [A-Za-z]+ [A-Za-z]+:", re.IGNORECASE)
chhh=pattern.match(joinedChat)
print(chhh)
joinedChat=re.sub(r"\[[^\]]*\]", '\n', joinedChat)
joinedChat=re.sub(r"Elif Yavrum:", '', joinedChat)
joinedChat=re.sub(r"Dogu Tamgac:", '', joinedChat)
joinedChat=joinedChat.replace("\u200e", "")
print(joinedChat[:500])

[04.10.17, 17:32:35] Elif Yavrum: ‎Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. [04.10.17, 17:32:35] Elif Yavrum: Numaram degisti ekle beni bro [04.10.17, 18:14:23] Dogu Tamgac: Ekledim bro [05.10.17, 12:14:44] Elif Yavrum: Kk [11.10.17, 21:24:19] Dogu Tamgac: Telefon aldim ha [11.10.17, 21:24:21] Dogu Tamgac: Ekle [11.10.17, 22:04:03] Elif Yavrum: Tamam eklerim ha [11.10.17, 22:12:36] Dogu Tamgac: Napan [11.10.17, 22:13
<re.Match object; span=(0, 33), match='[04.10.17, 17:32:35] Elif Yavrum:'>

  Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. 
  Numaram degisti ekle beni bro 
  Ekledim bro 
  Kk 
  Telefon aldim ha 
  Ekle 
  Tamam eklerim ha 
  Napan 
  Selinlerle viski iciyoruz 
  Sen 
  Ben de kayralardayim 
  Takiliyoz 
  Dersler basladi mi 
  Yok 
  Hala mathe vorkurs var 
  Sizde basladi mi 
  Basladi 
  Kan aliyolar :D 
  Ahahahah

In [108]:
from nltk.tokenize import word_tokenize

In [109]:
AI_tokens=word_tokenize(joinedChat)
print(AI_tokens[:500])

['Messages', 'and', 'calls', 'are', 'end-to-end', 'encrypted', '.', 'No', 'one', 'outside', 'of', 'this', 'chat', ',', 'not', 'even', 'WhatsApp', ',', 'can', 'read', 'or', 'listen', 'to', 'them', '.', 'Numaram', 'degisti', 'ekle', 'beni', 'bro', 'Ekledim', 'bro', 'Kk', 'Telefon', 'aldim', 'ha', 'Ekle', 'Tamam', 'eklerim', 'ha', 'Napan', 'Selinlerle', 'viski', 'iciyoruz', 'Sen', 'Ben', 'de', 'kayralardayim', 'Takiliyoz', 'Dersler', 'basladi', 'mi', 'Yok', 'Hala', 'mathe', 'vorkurs', 'var', 'Sizde', 'basladi', 'mi', 'Basladi', 'Kan', 'aliyolar', ':', 'D', 'Ahahahahahahha', 'Cidden', 'o', 'kadar', 'zor', 'mu', 'Kiz', 'var', 'mi', 'esasss', 'Kiz', 'yok', 'Net', 'cevap', ':', 'D', 'Ciddnenmi', ':', '(', 'Konular', 'zor', 'degil', 'de', 'Bende', 'her', 'yerde', 'erkek', 'kayniyo', ':', 'D', 'Siz', 'bunu', 'lisede', 'ogrendiniz', 'diyip', 'geciyo', 'Ama', 'ogrenmedik', ':', 'D', 'Haaa', 'O', 'kotuymus', 'bak', 'Burda', 'yuzde', '32', 'kiz', 'var', 'Aynen', 'Ama', 'calistim', 'bugun', 'Ohh', '

In [112]:
from nltk.probability import FreqDist
fdist=FreqDist()
for word in AI_tokens:
    fdist[word.lower()]+=1
print(fdist)

<FreqDist with 55288 samples and 343219 outcomes>
